In [133]:
import torch
import numpy as np

In [134]:
prediction = np.load('output52/000000.npy')

In [135]:
prediction = torch.from_numpy(prediction)

In [136]:
B, abC, H, W = prediction.size()
KA = 5
NC = 20
prediction = prediction.permute(0, 2, 3, 1).contiguous().view(B, -1, abC)

conf_pred = prediction[..., :KA].contiguous().view(B, -1, 1)
cls_pred = prediction[..., 1*KA : (1+NC)*KA].contiguous().view(B, -1, NC)
txtytwth_pred = prediction[..., (1+NC)*KA:].contiguous().view(B, -1, 4)

In [137]:
conf_pred = conf_pred[0]            #[H*W*KA, 1]
cls_pred = cls_pred[0]              #[H*W*KA, NC]
reg_pred = txtytwth_pred[0]    #[H*W*KA, 4]

In [138]:
reg_pred.shape

torch.Size([845, 4])

In [139]:
max_conf, max_conf_idx = torch.max(conf_pred, dim=0)

In [140]:
max_conf_cls = cls_pred[max_conf_idx]

In [141]:
VOC_CLASSES = (  # always index 0
    'aeroplane', 'bicycle', 'bird', 'boat',
    'bottle', 'bus', 'car', 'cat', 'chair',
    'cow', 'diningtable', 'dog', 'horse',
    'motorbike', 'person', 'pottedplant',
    'sheep', 'sofa', 'train', 'tvmonitor')

In [142]:
_, cls_result = torch.max(max_conf_cls, dim=1)

In [143]:
cls_result

tensor([11])

In [144]:
VOC_CLASSES[11]

'dog'

In [145]:
max_conf

tensor([-0.0004])

In [146]:
conf_pred.shape

torch.Size([845, 1])

In [147]:
scores = (torch.sigmoid(conf_pred) * torch.softmax(cls_pred, dim=-1)).flatten()

In [148]:
num_topk = min(100, reg_pred.size(0))
predicted_prob, topk_idxs = scores.sort(descending=True)
topk_scores = predicted_prob[:num_topk]
topk_idxs = topk_idxs[:num_topk]

In [149]:
keep_idxs = topk_scores > 0.1
scores = topk_scores[keep_idxs]
topk_idxs = topk_idxs[keep_idxs]

In [150]:
scores

tensor([0.4779, 0.4504, 0.3751, 0.1919, 0.1833])

In [ ]:
keep = np.zeros(len(bboxes), dtype=np.int64)
    for i in range(self.num_classes):
        inds = np.where(labels == i)[0]
        if len(inds) == 0:
            continue
        c_bboxes = bboxes[inds]
        c_scores = scores[inds]
        c_keep = self.nms(c_bboxes, c_scores)
        keep[inds[c_keep]] = 1